In [38]:
import pandas as pd
import numpy as np
from scipy.spatial import distance
from os import listdir, environ
from os.path import isfile, join
from pathlib import Path
import scipy.stats as stats
from tqdm import tqdm
from sklearn.datasets import make_regression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression, mutual_info_regression
from scipy.stats import rankdata
import warnings
warnings.filterwarnings('ignore')

In [39]:
path_election = '../../data/ronaldo/data/TSE/RS/data/aggregated_by_weighting_area.csv'
path_matrix = '../../data/ronaldo/data/IBGE/adjacency_matrices/weighting_areas/queen.csv'

In [40]:
data_election = pd.read_csv(path_election)
adjacency_matrix = pd.read_csv(path_matrix)
adjacency_matrix.set_index('Unnamed: 0', inplace=True)
adjacency_matrix.sort_index(inplace=True)

#data_election = data_election[data_election['NM_MUNICIPIO']=='PORTO ALEGRE']
#data_election = data_election[data_election['NM_MUNICIPIO']=='RIO GRANDE']
#data_election = data_election[data_election['NM_MUNICIPIO']=='CAXIAS DO SUL']
data_election.head()

,Cod_ap,SG_ UF,NM_MUNICIPIO,CD_MUNICIPIO,COD_LOCALIDADE_IBGE,NR_ZONA,NR_LOCAL_VOTACAO,local_unico,NR_SECAO,rural,...,lev_dist,QT_APTOS,QT_COMPARECIMENTO,QT_ABSTENCOES,QT_ELEITORES_BIOMETRIA_NH,Branco,Nulo,JAIR BOLSONARO,FERNANDO HADDAD,precision
0,4.300034e+12,RS,ACEGUÁ,88986,4300034.0,"[7, 7, 7]","[1023, 1040, 1058]","ESCOLA BARAO DO ACEGUA, AV. INTERNACIONAL, 33...","['[191, 246, 341, 55, 74, 85]', '[150, 269, 30...",False,...,1.0,3653,2974,679,655.0,109,101,1739,1025,"['TSE', 'TSE', 'TSE']"
1,4.300059e+12,RS,SANTA CECÍLIA DO SUL,89247,4316733.0,"[100, 100, 100, 100, 100, 100, 100, 100, 100, ...","[1015, 1171, 1180, 1198, 1201, 1244, 1660, 167...","SALAO COMUNITARIO, CAPELA SAO MARCOS, SANTA CE...","['[46]', '[44]', '[16]', '[26]', '[30]', '[45,...",False,...,1.0,3062,2682,380,439.0,68,129,1717,768,"['TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TS..."
2,4.300109e+12,RS,AGUDO,85014,4300109.0,"[119, 119, 119, 119, 119, 119, 119, 119, 119, ...","[1015, 1031, 1040, 1074, 1090, 1112, 1139, 114...","PAVILHÃO DA COMUNIDADE SÃO JOSÉ, RINCÃO DO PIN...","['[19, 36]', '[20]', '[27]', '[30, 46]', '[10]...",False,...,1.0,11908,10898,1010,814.0,360,546,6043,3949,"['TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TS..."
3,4.300208e+12,RS,AJURICABA,85030,4300208.0,"[23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23]","[1023, 1040, 1074, 1090, 1112, 1171, 1180, 120...","ESCOLA MUNICIPAL E.F. NELCI TOBIAS OEDMANN, RU...","['[14, 29, 305, 31, 6]', '[18, 28, 30, 32, 5]'...",False,...,1.0,5883,5009,874,573.0,87,171,3116,1635,"['TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TS..."
4,4.300307e+12,RS,ALECRIM,85057,4300307.0,"[102, 102, 102, 102, 102, 102, 102, 102, 102, ...","[1015, 1023, 1031, 1040, 1058, 1066, 1074, 108...","ESCOLA ESTADUAL DE ENS. FUND. ALECRIM, RUA PRI...","['[15, 35]', '[16, 33, 70]', '[21]', '[2]', '[...",False,...,1.0,5181,4587,594,1113.0,94,229,2355,1909,"['TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TS..."


In [41]:
data_election.set_index('Cod_ap', inplace=True)
data_election.sort_index(inplace=True)

In [42]:
votes = data_election[['JAIR BOLSONARO', 'FERNANDO HADDAD']].copy()
votes['FERNANDO HADDAD'] = votes['FERNANDO HADDAD'] / (votes['FERNANDO HADDAD'] + votes['JAIR BOLSONARO'])
votes['JAIR BOLSONARO'] = 1 - votes['FERNANDO HADDAD']
votes.head()

,JAIR BOLSONARO,FERNANDO HADDAD
Cod_ap,,
4.300034e+12,0.629161,0.370839
4.300059e+12,0.690946,0.309054
4.300109e+12,0.604784,0.395216
4.300208e+12,0.655862,0.344138
4.300307e+12,0.552298,0.447702


In [43]:
feature_votes = np.array(votes['JAIR BOLSONARO'].values).reshape(-1,1)
votes_sm = distance.pdist(feature_votes,'cityblock')
votes_sm = pd.DataFrame(distance.squareform(votes_sm), index=votes.index, columns= votes.index)
votes_sm.head()

Cod_ap,4.300034e+12,4.300059e+12,4.300109e+12,4.300208e+12,4.300307e+12,4.300406e+12,4.300406e+12,4.300406e+12,4.300406e+12,4.300455e+12,...,4.323309e+12,4.323358e+12,4.323408e+12,4.323457e+12,4.323507e+12,4.323606e+12,4.323705e+12,4.323754e+12,4.323770e+12,4.323804e+12
Cod_ap,,,,,,,,,,,,,,,,,,,,,
4.300034e+12,0.000000,0.061785,0.024377,0.026701,0.076862,0.081070,0.161790,0.078079,0.081176,0.110730,...,0.198699,0.109995,0.076722,0.128553,0.173661,0.199173,0.105505,0.141271,0.114843,0.113890
4.300059e+12,0.061785,0.000000,0.086162,0.035084,0.138647,0.142855,0.223575,0.139864,0.142961,0.172515,...,0.136914,0.048210,0.014937,0.190338,0.235446,0.137388,0.167290,0.203056,0.053058,0.052105
4.300109e+12,0.024377,0.086162,0.000000,0.051078,0.052486,0.056694,0.137414,0.053702,0.056799,0.086353,...,0.223076,0.134372,0.101099,0.104176,0.149284,0.223550,0.081128,0.116895,0.139220,0.138266
4.300208e+12,0.026701,0.035084,0.051078,0.000000,0.103564,0.107772,0.188492,0.104781,0.107877,0.137431,...,0.171998,0.083294,0.050020,0.155254,0.200362,0.172471,0.132206,0.167973,0.088142,0.087188
4.300307e+12,0.076862,0.138647,0.052486,0.103564,0.000000,0.004208,0.084928,0.001217,0.004313,0.033868,...,0.275561,0.186858,0.153584,0.051690,0.096798,0.276035,0.028642,0.064409,0.191706,0.190752


In [44]:
wa_remove = [x for x in adjacency_matrix.index if x not in votes_sm.index]
for wa in wa_remove:
    adjacency_matrix.drop(wa, axis = 0, inplace=True)
    adjacency_matrix.drop(str(wa), axis = 1, inplace=True)
adjacency_matrix.shape

(825, 825)

In [45]:
votes_sm = pd.DataFrame(votes_sm.values*adjacency_matrix.values, columns=votes_sm.index, index=votes_sm.index)
votes_sm.head()

Cod_ap,4.300034e+12,4.300059e+12,4.300109e+12,4.300208e+12,4.300307e+12,4.300406e+12,4.300406e+12,4.300406e+12,4.300406e+12,4.300455e+12,...,4.323309e+12,4.323358e+12,4.323408e+12,4.323457e+12,4.323507e+12,4.323606e+12,4.323705e+12,4.323754e+12,4.323770e+12,4.323804e+12
Cod_ap,,,,,,,,,,,,,,,,,,,,,
4.300034e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4.300059e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.04821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4.300109e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4.300208e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4.300307e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
votes_strangness = votes_sm.sum(axis=0)
#votes_strangness.sort_values(ascending=False, inplace=True)
rank_votes = rankdata(-votes_strangness, method='ordinal')

In [47]:
path_census = '../../data/ronaldo/data/IBGE/census_2010/RS/aggregated_by_weighting_area/joined/'
filenames = [filename for filename in listdir(path_census) if isfile(join(path_census, filename))]
list_df = []
for file in filenames:
    census_data = pd.read_csv(path_census+file)
    census_data.Cod_ap = census_data.Cod_ap.astype('int64')
    census_data.set_index('Cod_ap', inplace=True)
    census_data.sort_index(inplace=True)
    census_data.drop('CD_GEOCODM', axis=1, inplace=True)
    census_data.drop('NM_MUNICIP', axis=1, inplace=True)
    columns = census_data.columns.values.tolist()
    census_data.columns = [file.split('.')[0] + '_' + col_name for col_name in columns]
    list_df.append(census_data)

census_data = pd.concat(list_df, axis=1)
census_data.head()



,Alfabetizacao_01_V001,Alfabetizacao_01_V002,Alfabetizacao_01_V003,Alfabetizacao_01_V004,Alfabetizacao_01_V005,Alfabetizacao_01_V006,Alfabetizacao_01_V007,Alfabetizacao_01_V008,Alfabetizacao_01_V009,Alfabetizacao_01_V010,...,Responsavel_02_V207,Responsavel_02_V208,Responsavel_02_V209,Responsavel_02_V210,Responsavel_02_V211,Responsavel_02_V212,Responsavel_02_V213,Responsavel_02_V214,Responsavel_02_V215,Responsavel_02_V216
Cod_ap,,,,,,,,,,,,,,,,,,,,,
4300034001001,3841,19,50,61,87,75,79,86,89,90,...,119,119,133,117,96,79,41,53,31,19
4300059001001,3277,25,40,38,46,50,61,62,62,67,...,97,111,125,110,96,71,56,25,15,14
4300109001001,14658,65,119,166,183,233,270,269,217,254,...,389,451,476,409,364,297,179,150,84,65
4300208001001,6417,21,38,77,68,77,107,76,112,115,...,188,223,242,185,169,132,111,99,53,38
4300307001001,6204,11,48,67,77,89,104,109,113,113,...,144,186,244,222,196,224,149,91,53,34


In [48]:
census_data.drop(wa_remove, axis=0, inplace=True)
census_data=(census_data-census_data.min())/(census_data.max()-census_data.min())
census_data.fillna(0,inplace=True)
#Feature Selection Filtering
fs = SelectKBest(score_func=f_regression, k=43)
f_selected = fs.fit(census_data, votes_strangness)
cols = fs.get_support(indices=True)
f_selected = census_data.iloc[:,cols]
print(f_selected.columns.values[0])
#attributes.append(f_selected.columns.values[0])
#
census_sm = distance.pdist(f_selected,'cityblock')
census_sm = pd.DataFrame(distance.squareform(census_sm), index=census_data.index, columns= census_data.index)
census_sm = pd.DataFrame(census_sm.values*adjacency_matrix.values, columns=census_sm.index, index=census_sm.index)
census_strangness = census_sm.sum(axis=0)
sorted_rank = rankdata(-census_strangness, method='ordinal')
tau, p_value = stats.weightedtau(rank_votes, sorted_rank)
tau

Entorno_01_V005


0.3716776522017361

In [49]:
#kendal_rank_cor = dict()
#spearman_rank_cor = dict()
#for file, attr in zip(census_rank,attributes):
#    tau, p_value = stats.weightedtau(rank_votes, census_rank[file])
#    rho, pval = stats.spearmanr(rank_votes, census_rank[file])
#    kendal_rank_cor[file] = [attr,tau]
#    spearman_rank_cor[file] = [rho,pval]

In [50]:
#kendal_rank_cor = pd.DataFrame.from_dict(kendal_rank_cor,orient='index')
#kendal_rank_cor